<a href="https://colab.research.google.com/github/ronaldexim/public_transport/blob/master/public_transport_meeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade tables                                                              # potrzebne do wczytania zawartości  plików hdf5 do colaba

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


### Należy pobrać udostępnioną zawartość do katalogu na swoim Google Drive. 
### public_transport.7z to wersja spakowana plików z danymi.
### Następnie łączymy Colaba z własnym Google Drive - ikonka katalogu po lewej stronie i "Mount Drive".
### Ścieżkę w następnej komórce należy dopasować to katalogu gdzie wgrano pliki.



In [0]:
cd "/content/drive/My Drive/public_transport"

/content/drive/My Drive/public_transport


In [0]:
import pandas as pd
import numpy as np
import pickle
import urllib.request, json
import datetime
import geopy.distance
from tqdm import tqdm

apikey = ''

In [0]:
stops_df = pd.read_hdf('public_transport_warsaw.h5', key='stops_df_4', mode='r')
lines_df = pd.read_hdf('public_transport_warsaw.h5', key='lines_df', mode='r')
schedule_df = pd.read_hdf('public_transport_warsaw2.h5', key='schedule_df_2', mode='r')
GPS_df = pd.read_hdf('public_transport_warsaw3.h5', key='GPS_df_1', mode='r')
print(stops_df.shape, lines_df.shape, schedule_df.shape, GPS_df.shape)

(7727, 19) (345, 3) (918719, 13) (1160129, 13)


In [0]:
GPS_df.dtypes

Lines                    object
VehicleNumber            object
Brigade                  object
Time             datetime64[ns]
Lon                     float64
Lat                     float64
res_type                 object
qTime            datetime64[ns]
d_time                    int64
d_dist                  float64
coord                    object
uniqID                   object
vehID                    object
dtype: object

In [0]:
GPS_df.tail()


,Lines,VehicleNumber,Brigade,Time,Lon,Lat,res_type,qTime,d_time,d_dist,coord,uniqID,vehID
1160124,14,3824,3,2020-02-12 10:54:39,20.981739,52.209846,2,2020-02-12 10:55:01,10,0.00000,"(52.209846, 20.981739)",14_3824_3_2020-02-12 10:54:39,14_3824_3
1160125,13,3827,1,2020-02-12 10:54:40,20.966497,52.233574,2,2020-02-12 10:55:01,10,0.02620,"(52.233574, 20.966497)",13_3827_1_2020-02-12 10:54:40,13_3827_1
1160126,6,3828,10,2020-02-12 10:54:41,20.989725,52.265366,2,2020-02-12 10:55:01,10,0.00068,"(52.265366, 20.989725)",6_3828_10_2020-02-12 10:54:41,6_3828_10
1160127,6,3829,9,2020-02-12 10:54:41,21.024784,52.259600,2,2020-02-12 10:55:01,10,0.13949,"(52.2596, 21.024784)",6_3829_9_2020-02-12 10:54:41,6_3829_9
1160128,6,3830,5,2020-02-12 10:54:42,21.069824,52.246128,2,2020-02-12 10:55:01,10,0.00026,"(52.246128000000006, 21.069824)",6_3830_5_2020-02-12 10:54:42,6_3830_5


In [0]:
stop_coords = stops_df[stops_df.actual].set_index('id_nu')[['coord']].to_dict('dict')['coord']

In [0]:
# klucz (linia, brygada), zawiera listy z kolejnymi przystankami i planowanymi godzinami przyjazdu, czas i odległość od początku trasy

with open('schedule_dict.pickle', 'rb') as fp:
    schedule_dict = pickle.load(fp)

In [0]:
schedule_dict[('122', '4')]

In [0]:
def filter_gps(list_od_tuples, x_limits, y_limits):
    return [x for x in list_od_tuples if coord_check(x, x_limits, y_limits)] # (20.0, 22.0), (51.0, 53.0)
#     return list(filter(lambda x: coord_check(x, x_limits, y_limits), list_od_tuples))                    # faster?

def coord_check(to_check, x_limits, y_limits):
    x, y = to_check
    x_min, x_max = x_limits
    y_min, y_max = y_limits
    if((x_min < x < x_max) & (y_min < y < y_max)):
        return True
    else:
        return False
    
def flip(coords):
    return [(y, x) for x, y in coords]   # [x[::-1] for x in coords]

def simplify_to_nr(path, nr, tolerance=0, func=simplify_coords, max_iter=100): # simplify_coords_vw
    range_l = 0.0
    range_h = 1.0
    iteration = 1
    while True:
        param = (range_l + range_h)/2
        result = func(path, param)
        result_len = len(result)
        if((result_len >= nr-tolerance) & (result_len <= nr+tolerance)):
            print('Iteration: ', iteration, '      Number: ', result_len, ' Parameter: ', param)
            return result
        elif (result_len < nr): # parameter to high
            range_h = param
            iteration += 1
        else:                      # parameter to low
            range_l = param
            iteration += 1
        if(iteration > max_iter):
            print('calculation to long, Number: ', result_len, ' Parameter: ', param)
            return result  

def draw_simpl_path(line, brig_idx, nr_points, tolerance=0, rdp=True, wv=True):
    brigades = GPS_df[GPS_df.Lines.isin(['122'])].Brigade.value_counts().keys().tolist()
    print(line, brigades)
    line_stops = stops_coords_list([line])
    route = GPS_df[GPS_df.Lines.isin([line]) & 
                   GPS_df.Brigade.isin([brigades[brig_idx]])].sort_values(['Time'])[['Lon', 'Lat']].values
    route = filter_gps(route, (20.0, 22.0), (51.0, 53.0))
    route = np.ascontiguousarray(route)
    simpl_rdp = simplify_to_nr(route, nr_points)
    simpl_vw = simplify_to_nr(route, nr_points, tolerance, func=simplify_coords_vw)
    print(len(line_stops), route.shape, simpl_rdp.shape, simpl_vw.shape)
    print('stops ', path_dist(line_stops), 
          ' gps ', path_dist(route), 
          ' rdp ', path_dist(simpl_rdp), 
          ' wv', path_dist(simpl_vw))
    plt.figure(figsize=(10,6))
    plt.scatter(*zip(*route))
    if(rdp):
        plt.scatter(*zip(*simpl_rdp))
    if(wv):
        plt.scatter(*zip(*simpl_vw))
    plt.show()

    def path_dist(to_check_distance):
    sumed = 0.0
    last_first, last_second = to_check_distance[0]
    for first, second in to_check_distance:
        dist = round(geopy.distance.distance((last_second, last_first), (second, first)).kilometers, 5)
        sumed += dist
        last_first, last_second = first, second
    return sumed

def stops_coords_list(line):
    a = lines_df[lines_df.line_nr.isin(line)]['stops'].values[0]
    b = [stop_coords[x] for x in a]
    b = flip(b)          
    b = filter_gps(b, (20.0, 22.0), (51.0, 53.0))
    return b

def draw_stops(b):         
    #b = stops_coords_list(line)
    print('stops nr: ', len(b))
    plt.figure(figsize=(10,6))
    plt.scatter(*zip(*b)); 

### Ramer–Douglas–Peucker or Visvalingam-Whyatt algorithms



In [0]:
print(GPS_df.shape)
with open('GPS_df.pickle', 'rb') as fp:
    GPS_df = pickle.load(fp)
GPS_df.shape

(1160129, 13)


(1136854, 18)

In [0]:
with open('stops_dist_dict.pickle', 'rb') as fp:
    stops_dist_dict = pickle.load(fp)

In [0]:
len(stops_dist_dict)         # odległości w lini prostej między kolejnymi przystankami - wyszukiwanie w słowniku jest szybsze niż obliczanie z geopy, klucz: wartość -> ('6074_02', '6073_02'): 0.42795

10718